Download permissively licensed GitScehmas dataset and transform froma diuctionary to an array to facilitate loading into Neo4j

In [1]:
import requests
import json
 
def fetch_gitScehmas():
    # This is likely a time limited URL, preview link to follow and get download link from here: https://drive.google.com/file/d/1HwxeMQ7dym5md5igG2EBV0oPPR5Y0dAn/view?usp=sharing
    gitSchemasPermissiveDataset = 'https://drive.google.com/u/0/uc?id=1HwxeMQ7dym5md5igG2EBV0oPPR5Y0dAn&export=download'

    output_file = 'data/gitSchemasPermissiveDataset.json'
    transformed_file = 'data/gitSchemasPermissiveDataset_transformed.json'

    try:
        # Send a GET request to the URL
        response = requests.get(gitSchemasPermissiveDataset)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Access the content as text
            data = response.text

            # Write the data to a file
            with open(output_file, 'w', encoding='utf-8') as file:
                file.write(data)

            print(f"Data successfully written to {output_file}")
        else:
            print(f"Request failed with status code: {response.status_code}")

    except requests.exceptions.RequestException as e:
        print(f"Request exception: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

    
    with open(output_file, 'r', encoding='utf-8') as in_file:
        dict = json.load(in_file)
        array = [{'key': k, 'value': dict[k]} for k in dict]
        with open(transformed_file, 'w') as out_file:
            json.dump(array, out_file)

fetch_gitScehmas()

Data successfully written to data/gitSchemasPermissiveDataset.json


Load into Neo4j

In [9]:
# TO DELETE?
# import pandas as pd

# # File path to your CSV file


# # Read the CSV file into a DataFrame using pandas
# detected_keys = pd.read_csv('./data/test/detected_keys.csv')

# testSchemas = []

# for index, row in detected_keys.iterrows():
#     testSchemas.append(row['id'])


In [3]:
# Move tansformed file to import directory of your neo4j instance before running the below
# Ensure that apoc.import.file.enabled=true us set in your apoc.conf
from Neo4jDriver import Neo4jDriver


with Neo4jDriver() as neo4j:

    # Create necessary schema
    createSchemas = ["CREATE CONSTRAINT IF NOT EXISTS FOR (s:Schema) REQUIRE (s.url) IS UNIQUE;", 
        "CREATE CONSTRAINT IF NOT EXISTS FOR (t:Table) REQUIRE (t.name, t.schemaId) IS UNIQUE;", 
        "CREATE CONSTRAINT IF NOT EXISTS FOR (c:Column) REQUIRE (c.name, c.table, c.type, c.schemaId) IS UNIQUE;", 
        "CREATE CONSTRAINT IF NOT EXISTS FOR (p:PrimaryKey) REQUIRE (p.table, p.schemaId) IS UNIQUE;", 
        "CREATE CONSTRAINT IF NOT EXISTS FOR (f:ForeignKey) REQUIRE (f.table, f.schemaId, f.fkName) IS UNIQUE;"]
    
    for q in createSchemas:
        neo4j.execute_write_query(q)


    # Load from json file
    file = '///gitSchemasPermissiveDataset_transformed.json'

    returning_statement = f"""
        CALL apoc.load.json('{file}')
        YIELD value AS schema
        RETURN schema
        """
    # WHERE schema.INFO.url IN $testSchemas
    per_item_statement = """
        // Create Schema
        WITH schema
        WITH schema.value.INFO.url AS key, schema.value AS schema
        MERGE (schemaNode:Schema { url: key })
        SET schemaNode+= schema.INFO
        WITH schemaNode, schema
        // Create associated Tables
        WITH keys(schema.TABLES) AS tableKeys, schemaNode, schema
        UNWIND tableKeys AS tableKey
        MERGE (tableNode:Table { name: tableKey, schemaId: schemaNode.url })<-[:CONTAINS_TABLE]-(schemaNode)
        // Create linked Columns
        WITH schema.TABLES[tableKey] AS table, tableNode, schemaNode, tableKey
        WITH tableNode, table, table.COLUMNS AS columns, schemaNode, tableKey
        UNWIND columns AS column
        MERGE (columnNode:Column { name: column[0], table: tableKey, type: column[1], schemaId:schemaNode.url })<-[:HAS_COLUMN]-(tableNode)
        // Create Primary Keys
        WITH tableNode, table, table.PRIMARY_KEYS AS primaryKeys, schemaNode
        UNWIND primaryKeys AS primaryKey
        MATCH (tableNode)-[:HAS_COLUMN]-(pkc:Column { name: primaryKey })
        MERGE (pk:PrimaryKey { table:tableNode.name, schemaId:schemaNode.url })
        MERGE (pkc)<-[:PK_COLUMN]-(pk)
        // Create Foreign Keys
        WITH tableNode, table.FOREIGN_KEYS AS foreignKeys, schemaNode
        UNWIND foreignKeys AS foreignKey
        UNWIND foreignKey.FOREIGN_KEY AS fk_component
        MATCH (tableNode)-[:HAS_COLUMN]-(fkc:Column { name: fk_component })
        MERGE (fk:ForeignKey { table:tableNode.name, schemaId:schemaNode.url, fkName:foreignKey.FOREIGN_KEY })
        MERGE (fkc)<-[:FK_COLUMN]-(fk)
        WITH foreignKey, schemaNode, tableNode, fk
        UNWIND foreignKey.REFERENCE_COLUMN AS fk_reference_column
        MATCH (schemaNode)-[:CONTAINS_TABLE]->(:Table { name: foreignKey.REFERENCE_TABLE })-[:HAS_COLUMN]->(fkrc:Column {name: fk_reference_column})
        MERGE (fkrc)<-[:FK_REFERENCE_COLUMN]-(fk)
        """
    
    options = "{ batchSize:5, parallel: true}" #, params: {testSchemas: " + str(testSchemas) + "} 



    load_query = f"""
        CALL apoc.periodic.iterate("{returning_statement}","{per_item_statement}",{options})
    """

    neo4j.execute_write_query(load_query)


Connected to Neo4j database!
Connection to Neo4j closed.
